## Reaction wheel static and dynamic imbalance demo ##

### Documentation

#### Reaction wheel frames

The current implementation introduces a reaction wheel static frame and a reaction wheel spin frame with both frames' +z pointing in the same direction as spin axis in body frame, and +x, +y orthogonal to the +z basis vector solved using the Gram-Schmidt process. Both frames are orthonomal. To define the static frame, we let,\
$u_1 = v_w$\
where $v_w$ is the normalized spin axis in body frame. Then we have two vectors defined to be used in the Gram-Schmidt process,\
$v_1 = \begin{pmatrix}0\\0\\1\end{pmatrix}$\
$v_2 = \begin{pmatrix}1\\0\\0\end{pmatrix}$\
if $v_w$ is collinear with $v_1$, then $v_2$ is used in the projection,\
$u_2 = v_2 - \frac{v_2\cdot u_1}{u_1\cdot u_1}u_1$\
which then simplifies to,\
$u_2 = v_2 - (v_2\cdot u_1)u_1$\
otherwise $v_1$ is used,\
$u_2 = v_1 - (v1\cdot u_1)u_1$\
$u_2$ is then normalized. The third basis vector is solved for using,\
$u_3 = u_1 \times u_2$\
$u_1$, $u_2$ and $u_3$ then forms the static frame basis,\
$W = \{u_1,u_2,u_3\}$\
The reaction wheel spin frame is defined to be the static frame rotated by the reaction wheel angle about the +z axis.

#### Static imbalance
The static imbalance force vector is modeled using the spin frame and points along the +x axis,\
$\vec{F_{s,spin}} = U_s\omega^2\hat{x}$\
where $U_s$ represents the static imbalance coefficient, $\omega$ represents the angular velocity of the reaction wheel, and $\hat{x}$ represents the unit vector that rotates with the reaction wheel. To tranform to the body frame, two transformations were applied: a transformation is applied first to get to the reaction wheel static frame,\
$\vec{F_{s,static}} = R\vec{F_{s,spin}}$\
where R represents the transformation between the reaction wheel spin frame and the static frame. A transformation is then applied to get from the static frame to the body frame,\
$\vec{F_{s,body}} = M\vec{F_{s,static}}$\
Where M represents the transformation between the reaction wheel static frame and body frame. To get the torque applied by this force to the spacecraft, a lever arm is solved using the center of mass (CoM) of the spacecraft and the coordinates of the reaction wheels (coord),\
$\vec{r} = \vec{r_{coord}} - \vec{r_{CoM}}$\
then the torque is solved using,\
$\vec{\tau_{s,body}} = \vec{r} \times \vec{F_{s,body}}$

#### Dynamic imbalance
The reaction wheel dynamic imbalance torque is modeled using,\
$\vec{\tau_{d,spin}} = U_d\omega^2\hat{x}$\
this is a vector pointing along +x in the reaction wheel spin frame and rotates with the wheel in the static frame. A transformation is applied to get from the reaction wheel spin frame to the static frame,\
$\vec{\tau_{d,static}} = R\vec{\tau_{d,spin}}$\
then the transformation between the spin frame and the body frame is applied to solve for the torque applied to the spacecraft,\
$\vec{\tau_{d,body}} = M\vec{\tau_{d,static}}$

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


from alea.sim.kernel.kernel import AleasimKernel


from alea.sim.kernel.root_models import AleasimRootModel
from alea.sim.spacecraft.eps.power_system import PowerSystemModel
from alea.sim.spacecraft.eps.solar_panel import SolarPanelModel
from alea.sim.epa.attitude_dynamics import AttitudeDynamicsModel
from alea.sim.epa.orbit_dynamics import OrbitDynamicsModel
from alea.sim.spacecraft.spacecraft import Spacecraft

from skyfield.api import utc
import datetime

### Setup and running the simulation

In [ ]:
#Change date here
kernel = AleasimKernel(dt=0.01,date=datetime.datetime(2024, 5, 17, tzinfo=utc))

#setting up kernel
root = AleasimRootModel(kernel)
kernel.add_model(root)


adyn:AttitudeDynamicsModel = kernel.get_model(AttitudeDynamicsModel)
odyn:OrbitDynamicsModel = kernel.get_model(OrbitDynamicsModel)
sc:Spacecraft = kernel.get_model(Spacecraft)
#change attitue dynamic state array here
adyn.set_state(np.array([0.0,1.0,0.0,0.0,0.0,0.0,0.0]))

#setting up reaction wheels and magnetorquers
rwx, rwy, rwz = sc.aocs._rws
mtqx, mtqy, mtqz =  sc.aocs._mtqs


mag_sens = sc.aocs._mag_sens
sun_sens = sc.aocs._sun_sens
gyro = sc.aocs._gyro_sens


power_sys = PowerSystemModel(kernel)
kernel.add_model(power_sys)


solar_panels: list[SolarPanelModel] = power_sys.solar_panels
eps = power_sys.eps


eps._force_state_of_charge(50)

#setting the AOCS mode to nadir pointing
kernel.advance_n(2)
sc.aocs.set_mode(sc.aocs.AOCSMode.POINTING_CAMERA_NADIR)

#running the simulation
kernel.advance(125)

### Plots
#### Spacecraft attitude dynamic plots

In [ ]:
objs = plt.plot(adyn.time_array, adyn.state_array[:,0:4])
plt.title("Spacecraft Quaternions")
plt.legend(iter(objs),('q0','q1','q2','q3'))
plt.ylabel("Quaternions")
plt.xlabel("Time (s)")
plt.show()

objs = plt.plot(adyn.time_array, adyn.state_array[:,4:7])
plt.title("Spacecraft Angular Velocities")
plt.legend(iter(objs),('w_x','w_y','w_z'))
plt.ylabel("Angular velocity (rad/s)")
plt.xlabel("Time (s)")
plt.show()

#### Dynamic Imbalance plots

In [ ]:
plt.title("Reaction wheel dynamic imbalance torque magnitudes")
plt.plot(rwx.time_array, rwx.state_array[:,11])
plt.plot(rwy.time_array, rwy.state_array[:,11])
plt.plot(rwz.time_array, rwz.state_array[:,11])
plt.legend(('rw_x','rw_y','rw_z'))
plt.xlabel("Time(s)")
plt.ylabel("Torque (N*m)")
plt.show()

plt.title("Reaction wheel angular velocities")
plt.plot(rwx.time_array, rwx.state_array[:,5])
plt.plot(rwy.time_array, rwy.state_array[:,5])
plt.plot(rwz.time_array, rwz.state_array[:,5])
plt.legend(('rw_x','rw_y','rw_z'))
plt.xlabel("Time (s)")
plt.ylabel("Angular velocities (rad/s)")
plt.show()

plt.title("Reaction wheel rotation angle")
plt.plot(rwx.time_array, rwx.state_array[:,9])
plt.plot(rwy.time_array, rwy.state_array[:,9])
plt.plot(rwz.time_array, rwz.state_array[:,9])
plt.legend(('rw_x','rw_y','rw_z'))
plt.xlabel("Time (s)")
plt.ylabel("Angle (rad)")
plt.show()

#### Static imbalance plots

In [ ]:
plt.title("Reaction wheel static imbalance torque magnitudes")
plt.plot(rwx.time_array, rwx.state_array[:,10])
plt.plot(rwy.time_array, rwy.state_array[:,10])
plt.plot(rwz.time_array, rwz.state_array[:,10])
plt.legend(('rw_x','rw_y','rw_z'))
plt.xlabel("Time(s)")
plt.ylabel("Torque (N*m)")
plt.show()

plt.title("Reaction wheel angular velocities")
plt.plot(rwx.time_array, rwx.state_array[:,5])
plt.plot(rwy.time_array, rwy.state_array[:,5])
plt.plot(rwz.time_array, rwz.state_array[:,5])
plt.legend(('rw_x','rw_y','rw_z'))
plt.xlabel("Time (s)")
plt.ylabel("Angular velocities (rad/s)")
plt.show()

#### Net disturbalance torques plots and reaction wheel torques

In [ ]:
plt.title("Reaction wheel torque magnitudes")
plt.plot(rwx.time_array, rwx.state_array[:,0])
plt.plot(rwy.time_array, rwy.state_array[:,0])
plt.plot(rwz.time_array, rwz.state_array[:,0])
plt.legend(('rw_x','rw_y','rw_z'))
plt.xlabel("Time(s)")
plt.ylabel("Torque (N*m)")
plt.show()

plt.title("Net reaction wheel disturbalance torques")
objs = plt.plot(adyn.time_array, adyn.state_array[:,10:13])
plt.legend(iter(objs),('tau_x','tau_y','tau_z'))
plt.xlabel("Time (s)")
plt.ylabel("Torque (N*m)")
plt.show()

plt.title("Net disturbalance torques")
objs = plt.plot(adyn.time_array, adyn.state_array[:,13:16])
plt.legend(iter(objs),('tau_x','tau_y','tau_z'))
plt.xlabel("Time (s)")
plt.ylabel("Torque (N*m)")
plt.show()

#### AOCS performance plots

In [ ]:
aocs = sc.aocs

objs = plt.plot(aocs.time_array, aocs.state_array[:,8:11])
plt.legend(iter(objs), aocs.saved_state_element_names[8:11])
plt.title("Absolute Knowledge Error")
plt.xlabel('Time (s)')
plt.ylabel('degrees')
plt.show()

objs = plt.plot(aocs.time_array, aocs.state_array[:,11:14])
plt.legend(iter(objs), aocs.saved_state_element_names[11:14])
plt.title("Absolute Pointing Error")
plt.xlabel('Time (s)')
plt.ylabel('degrees')
plt.show()